<a href="https://colab.research.google.com/github/salsaimon/proyecto_recursoshumanos/blob/main/preprocesamientoSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#  pandas actualizado correctamente
!pip install --upgrade pandas

# importar pandas y sqlite3
import pandas as pd
import sqlite3 as sql

# Crear una conexión a la base de datos SQLite
conn = sql.connect('/content/drive/MyDrive/analitica3win/proyecto_recursoshumanos/base.db')

# Consultar las primeras filas de cada tabla
print("\nDatos de la tabla 'retiros':")
df_retiros = pd.read_sql("SELECT * FROM retiros LIMIT 5", conn)
print(df_retiros)

print("\nDatos de la tabla 'general':")
df_general = pd.read_sql("SELECT * FROM general LIMIT 5", conn)
print(df_general)

print("\nDatos de la tabla 'employee':")
df_employee = pd.read_sql("SELECT * FROM employee LIMIT 5", conn)
print(df_employee)

print("\nDatos de la tabla 'manager':")
df_manager = pd.read_sql("SELECT * FROM manager LIMIT 5", conn)
print(df_manager)

# Realizar consultas SQL para análisis
print("\nConteo de 'Attrition' en la tabla 'retiros':")
attrition_count = pd.read_sql("SELECT Attrition, COUNT(*) FROM retiros GROUP BY Attrition", conn)
print(attrition_count)

print("\nConteo de 'BusinessTravel' en la tabla 'general':")
business_travel_count = pd.read_sql("SELECT BusinessTravel, COUNT(*) FROM general GROUP BY BusinessTravel", conn)
print(business_travel_count)

# Realizar más consultas según sea necesario
# Ejemplo: Conteo de 'JobSatisfaction' en la tabla 'employee'
print("\nConteo de 'JobSatisfaction' en la tabla 'employee':")
job_satisfaction_count = pd.read_sql("SELECT JobSatisfaction, COUNT(*) FROM employee GROUP BY JobSatisfaction", conn)
print(job_satisfaction_count)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 57.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.2 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 2.2.2 which is incompatible.

Datos de la tabla 'retiros':
   index  EmployeeID Attrition       retirementDate retirementType  \
0      0           2       Yes  2016-10-15 00:00:00    Resignation   
1      1           7       Yes  2016-07-20 00:00:00    Resignation   
2      2          14       Yes  2016-11-08 00:00:00    Resignation   
3      3          29       Yes  2016-10-15 00:00:00    Resignation   
4      4          31   

In [3]:
# Obtener la lista de tablas en la base de datos
tables = [row[0] for row in conn.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()]

# Mostrar información sobre cada tabla y sus columnas
for table in tables:
    print(f"\nInformación sobre la tabla '{table}':")

    # Consultar las primeras filas de la tabla
    df = pd.read_sql(f"SELECT * FROM {table} LIMIT 5", conn)
    print(f"\nPrimeras 5 filas de la tabla '{table}':")
    print(df)

    # Mostrar la estructura de la tabla (tipos de datos)
    print(f"\nEstructura de la tabla '{table}':")
    cur = conn.cursor()
    cur.execute(f"PRAGMA table_info({table})")
    structure = cur.fetchall()
    for column in structure:
        print(f"Columna: {column[1]}, Tipo: {column[2]}")



Información sobre la tabla 'retiros':

Primeras 5 filas de la tabla 'retiros':
   index  EmployeeID Attrition       retirementDate retirementType  \
0      0           2       Yes  2016-10-15 00:00:00    Resignation   
1      1           7       Yes  2016-07-20 00:00:00    Resignation   
2      2          14       Yes  2016-11-08 00:00:00    Resignation   
3      3          29       Yes  2016-10-15 00:00:00    Resignation   
4      4          31       Yes  2016-02-08 00:00:00    Resignation   

  resignationReason  
0            Others  
1            Stress  
2            Others  
3            Others  
4            Salary  

Estructura de la tabla 'retiros':
Columna: index, Tipo: INTEGER
Columna: EmployeeID, Tipo: INTEGER
Columna: Attrition, Tipo: TEXT
Columna: retirementDate, Tipo: TIMESTAMP
Columna: retirementType, Tipo: TEXT
Columna: resignationReason, Tipo: TEXT

Información sobre la tabla 'general':

Primeras 5 filas de la tabla 'general':
   index  Age     BusinessTravel        

In [7]:
## tratamiento de la tabla de retiros
# Contar registros por ID de empleado en la tabla de retiros
employee_id_count = pd.read_sql("SELECT EmployeeID, COUNT(*) as count FROM retiros GROUP BY EmployeeID", conn)

# Mostrar el resultado
print("\nConteo de registros por ID de empleado en la tabla 'retiros':")
print(employee_id_count)






Conteo de registros por ID de empleado en la tabla 'retiros':
     EmployeeID  count
0             2      1
1             7      1
2            14      1
3            29      1
4            31      1
..          ...    ...
706        4382      1
707        4387      1
708        4389      1
709        4392      1
710        4403      1

[711 rows x 2 columns]


In [9]:
# Resumen de la columna 'retirementDate'
print("\nResumen de la columna 'retirementDate' en la tabla 'retiros':")

# Contar valores no nulos y nulos
total_retiros = pd.read_sql("SELECT COUNT(*) FROM retiros WHERE retirementDate IS NOT NULL", conn).iloc[0, 0]
total_nulos = pd.read_sql("SELECT COUNT(*) FROM retiros WHERE retirementDate IS NULL", conn).iloc[0, 0]
print(f"Total de fechas de retiro no nulas: {total_retiros}")
print(f"Total de fechas de retiro nulas: {total_nulos}")

# Estadísticas descriptivas
retiros_df = pd.read_sql("SELECT retirementDate FROM retiros WHERE retirementDate IS NOT NULL", conn)
retiros_df['retirementDate'] = pd.to_datetime(retiros_df['retirementDate'])  # Convertir a datetime
print("\nEstadísticas descriptivas de 'retirementDate':")
print(f"Fecha mínima de retiro: {retiros_df['retirementDate'].min()}")
print(f"Fecha máxima de retiro: {retiros_df['retirementDate'].max()}")

# Distribución por año
retiros_df['Año'] = retiros_df['retirementDate'].dt.year
retiros_por_anio = retiros_df.groupby('Año').size().reset_index(name='Cantidad de retiros')
print("\nDistribución de retiros por año:")
print(retiros_por_anio)




Resumen de la columna 'retirementDate' en la tabla 'retiros':
Total de fechas de retiro no nulas: 711
Total de fechas de retiro nulas: 0

Estadísticas descriptivas de 'retirementDate':
Fecha mínima de retiro: 2015-01-03 00:00:00
Fecha máxima de retiro: 2016-12-16 00:00:00

Distribución de retiros por año:
    Año  Cantidad de retiros
0  2015                   61
1  2016                  650

Distribución de retiros por mes:
        Mes  Cantidad de retiros
0   2015-01                    5
1   2015-02                    7
2   2015-03                    2
3   2015-04                    4
4   2015-05                    7
5   2015-06                    8
6   2015-07                    4
7   2015-08                    4
8   2015-09                    7
9   2015-10                    8
10  2015-11                    4
11  2015-12                    1
12  2016-01                   66
13  2016-02                   61
14  2016-03                   59
15  2016-04                   48
16  2016-0

In [10]:
# Eliminar los registros con fechas de retiro en el año 2015
conn.execute("DELETE FROM retiros WHERE strftime('%Y', retirementDate) = '2015'")
conn.commit()

# Verificar que los registros han sido eliminados
retiros_df = pd.read_sql("SELECT retirementDate FROM retiros WHERE retirementDate IS NOT NULL", conn)
retiros_df['retirementDate'] = pd.to_datetime(retiros_df['retirementDate'])  # Convertir a datetime
print("\nVerificación después de eliminar los datos del año 2015:")
print(f"Fecha mínima de retiro: {retiros_df['retirementDate'].min()}")
print(f"Fecha máxima de retiro: {retiros_df['retirementDate'].max()}")

# Distribución de retiros por año después de eliminar los datos
retiros_df['Año'] = retiros_df['retirementDate'].dt.year
retiros_por_anio = retiros_df.groupby('Año').size().reset_index(name='Cantidad de retiros')
print("\nDistribución de retiros por año después de eliminación:")
print(retiros_por_anio)


Verificación después de eliminar los datos del año 2015:
Fecha mínima de retiro: 2016-01-02 00:00:00
Fecha máxima de retiro: 2016-12-16 00:00:00

Distribución de retiros por año después de eliminación:
    Año  Cantidad de retiros
0  2016                  650


In [13]:
##analisis de retirementType

# Consultar los datos de la columna 'retirementType'
retiros_df = pd.read_sql("SELECT retirementType FROM retiros", conn)

# Contar valores únicos y frecuencia de cada tipo
print("\nResumen de la columna 'retirementType':")
print(f"Valores únicos en 'retirementType': {retiros_df['retirementType'].nunique()}")
print("\nFrecuencia de cada tipo de retiro:")
print(retiros_df['retirementType'].value_counts())

# Contar valores nulos
total_nulos = retiros_df['retirementType'].isnull().sum()
print(f"\nTotal de valores nulos en 'retirementType': {total_nulos}")


Resumen de la columna 'retirementType':
Valores únicos en 'retirementType': 2

Frecuencia de cada tipo de retiro:
retirementType
Resignation    585
Fired           65
Name: count, dtype: int64

Total de valores nulos en 'retirementType': 0


In [18]:
import pandas as pd

# Leer todos los datos de la tabla 'retiros'
retiros_df = pd.read_sql("SELECT * FROM retiros", conn)

# Crear una copia del DataFrame original para modificaciones
retiros_processed_df = retiros_df.copy()

# Codificación One-Hot para la columna 'retirementType' sin eliminar ninguna categoría
encoded_retiros_df = pd.get_dummies(retiros_processed_df['retirementType'], prefix='retirementType')

# Convertir los valores True/False a 0/1
encoded_retiros_df = encoded_retiros_df.astype(int)

# Agregar las columnas codificadas al DataFrame procesado
retiros_processed_df = pd.concat([retiros_processed_df, encoded_retiros_df], axis=1)

# Eliminar la columna original 'retirementType' si ya no es necesaria
retiros_processed_df.drop('retirementType', axis=1, inplace=True)

# Imprimir el DataFrame procesado
print(retiros_processed_df.head())

# Mostrar el tamaño del DataFrame procesado
print(f"\nNúmero total de filas en el DataFrame procesado: {len(retiros_processed_df)}")


   index  EmployeeID Attrition       retirementDate resignationReason  \
0      0           2       Yes  2016-10-15 00:00:00            Others   
1      1           7       Yes  2016-07-20 00:00:00            Stress   
2      2          14       Yes  2016-11-08 00:00:00            Others   
3      3          29       Yes  2016-10-15 00:00:00            Others   
4      4          31       Yes  2016-02-08 00:00:00            Salary   

   retirementType_Fired  retirementType_Resignation  
0                     0                           1  
1                     0                           1  
2                     0                           1  
3                     0                           1  
4                     0                           1  

Número total de filas en el DataFrame procesado: 650


In [19]:

# Leer todos los datos de la tabla 'retiros'
retiros_df = pd.read_sql("SELECT resignationReason FROM retiros", conn)

# Resumen de la columna 'resignationReason'
unique_reasons = retiros_df['resignationReason'].unique()
reason_counts = retiros_df['resignationReason'].value_counts()
missing_values = retiros_df['resignationReason'].isnull().sum()

# Imprimir resumen
print(f"\nValores únicos en 'resignationReason': {unique_reasons}")
print("\nFrecuencia de cada motivo de renuncia:")
print(reason_counts)
print(f"\nNúmero total de valores nulos en 'resignationReason': {missing_values}")


Valores únicos en 'resignationReason': ['Others' 'Stress' 'Salary' None]

Frecuencia de cada motivo de renuncia:
resignationReason
Others    296
Salary    173
Stress    116
Name: count, dtype: int64

Número total de valores nulos en 'resignationReason': 65


In [24]:
# Reemplazar valores nulos con 'Unknown'
retiros_processed_df['resignationReason'].fillna('Unknown', inplace=True)

# Codificación One-Hot para la columna 'resignationReason'
encoded_reasons_df = pd.get_dummies(retiros_processed_df['resignationReason'], prefix='resignationReason', drop_first=False)

# Convertir valores True/False a 1/0 en las columnas codificadas
encoded_reasons_df = encoded_reasons_df.astype(int)

# Agregar las columnas codificadas al DataFrame procesado
retiros_processed_df = pd.concat([retiros_processed_df, encoded_reasons_df], axis=1)

# Eliminar la columna original 'resignationReason'
retiros_processed_df.drop('resignationReason', axis=1, inplace=True)

# Eliminar la columna 'Unknown' resultante de la codificación
if 'resignationReason_Unknown' in retiros_processed_df.columns:
    retiros_processed_df.drop('resignationReason_Unknown', axis=1, inplace=True)

# Imprimir el DataFrame procesado
print(retiros_processed_df.head())

# Mostrar el tamaño del DataFrame procesado
print(f"\nNúmero total de filas en el DataFrame procesado: {len(retiros_processed_df)}")

KeyError: 'resignationReason'

In [22]:
# Imprimir las primeras filas del DataFrame procesado
print("Primeras filas de retiros_processed_df:")
print(retiros_processed_df.head())

# Mostrar el resumen de las nuevas columnas de codificación One-Hot
print("\nResumen de las columnas codificadas para 'resignationReason':")
encoded_columns = [col for col in retiros_processed_df.columns if col.startswith('resignationReason')]
print(retiros_processed_df[encoded_columns].sum())

# Mostrar el número total de filas en el DataFrame procesado
print(f"\nNúmero total de filas en el DataFrame procesado: {len(retiros_processed_df)}")

Primeras filas de retiros_processed_df:
   index  EmployeeID Attrition       retirementDate resignationReason  \
0      0           2       Yes  2016-10-15 00:00:00            Others   
1      1           7       Yes  2016-07-20 00:00:00            Stress   
2      2          14       Yes  2016-11-08 00:00:00            Others   
3      3          29       Yes  2016-10-15 00:00:00            Others   
4      4          31       Yes  2016-02-08 00:00:00            Salary   

   retirementType_Fired  retirementType_Resignation  
0                     0                           1  
1                     0                           1  
2                     0                           1  
3                     0                           1  
4                     0                           1  

Resumen de las columnas codificadas para 'resignationReason':
resignationReason    OthersStressOthersOthersSalaryOthersStressOthe...
dtype: object

Número total de filas en el DataFrame procesado:

In [25]:
# Reemplazar valores nulos con 'Unknown'
retiros_processed_df['resignationReason'].fillna('Unknown', inplace=True)

# Codificación One-Hot para la columna 'resignationReason'
encoded_reasons_df = pd.get_dummies(retiros_processed_df['resignationReason'], prefix='resignationReason', drop_first=False)

# Agregar las columnas codificadas al DataFrame procesado
retiros_processed_df = pd.concat([retiros_processed_df, encoded_reasons_df], axis=1)

# Eliminar la columna original 'resignationReason'
retiros_processed_df.drop('resignationReason', axis=1, inplace=True)

# Eliminar la columna 'Unknown' resultante de la codificación
if 'resignationReason_Unknown' in retiros_processed_df.columns:
    retiros_processed_df.drop('resignationReason_Unknown', axis=1, inplace=True)

# Imprimir el DataFrame procesado
print(retiros_processed_df.head())

# Mostrar el tamaño del DataFrame procesado
print(f"\nNúmero total de filas en el DataFrame procesado: {len(retiros_processed_df)}")

KeyError: 'resignationReason'

In [27]:
# Convertir valores True/False a 1/0 en las columnas codificadas
encoded_reasons_df = encoded_reasons_df.astype(int)

# Agregar las columnas codificadas al DataFrame procesado
retiros_processed_df = pd.concat([retiros_processed_df, encoded_reasons_df], axis=1)

# Eliminar la columna original 'resignationReason'
retiros_processed_df.drop('resignationReason', axis=1, inplace=True)

# Eliminar la columna 'Unknown' resultante de la codificación si existe
if 'resignationReason_Unknown' in retiros_processed_df.columns:
    retiros_processed_df.drop('resignationReason_Unknown', axis=1, inplace=True)

# Imprimir el DataFrame procesado
print(retiros_processed_df.head())

KeyError: "['resignationReason'] not found in axis"

In [32]:
# Convertir valores True/False a 1/0 en las columnas especificadas
columns_to_convert = ['resignationReason_Others', 'resignationReason_Salary', 'resignationReason_Stress']
retiros_processed_df[columns_to_convert] = retiros_processed_df[columns_to_convert].astype(int)

# Imprimir el DataFrame procesado para verificar
print(retiros_processed_df.head())

   index  EmployeeID Attrition       retirementDate  retirementType_Fired  \
0      0           2       Yes  2016-10-15 00:00:00                     0   
1      1           7       Yes  2016-07-20 00:00:00                     0   
2      2          14       Yes  2016-11-08 00:00:00                     0   
3      3          29       Yes  2016-10-15 00:00:00                     0   
4      4          31       Yes  2016-02-08 00:00:00                     0   

   retirementType_Resignation  resignationReason_Others  \
0                           1                         1   
1                           1                         0   
2                           1                         1   
3                           1                         1   
4                           1                         0   

   resignationReason_Salary  resignationReason_Stress  \
0                         0                         0   
1                         0                         1   
2          

<ipython-input-32-835aef14ce2e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retiros_processed_df[columns_to_convert] = retiros_processed_df[columns_to_convert].astype(int)


In [33]:
# Resumen de la columna 'Attrition'
attrition_summary = retiros_processed_df['Attrition'].value_counts()
total_count = len(retiros_processed_df)

# Imprimir el resumen
print("Resumen de la variable 'Attrition':")
print(attrition_summary)
print(f"\nNúmero total de registros en 'Attrition': {total_count}")
print(f"Proporción de cada categoría en 'Attrition':")
print(attrition_summary / total_count)

Resumen de la variable 'Attrition':
Attrition
Yes    650
Name: count, dtype: int64

Número total de registros en 'Attrition': 650
Proporción de cada categoría en 'Attrition':
Attrition
Yes    1.0
Name: count, dtype: float64


In [35]:
# Eliminar la columna 'Attrition' del DataFrame procesado
retiros_processed_df.drop('Attrition', axis=1, inplace=True)

# Imprimir el DataFrame procesado para verificar los cambios
print(retiros_processed_df.head())

# Mostrar el tamaño del DataFrame procesado
print(f"\nNúmero total de filas en el DataFrame procesado: {len(retiros_processed_df)}")

   index  EmployeeID       retirementDate  retirementType_Fired  \
0      0           2  2016-10-15 00:00:00                     0   
1      1           7  2016-07-20 00:00:00                     0   
2      2          14  2016-11-08 00:00:00                     0   
3      3          29  2016-10-15 00:00:00                     0   
4      4          31  2016-02-08 00:00:00                     0   

   retirementType_Resignation  resignationReason_Others  \
0                           1                         1   
1                           1                         0   
2                           1                         1   
3                           1                         1   
4                           1                         0   

   resignationReason_Salary  resignationReason_Stress  \
0                         0                         0   
1                         0                         1   
2                         0                         0   
3             

<ipython-input-35-bf67eeb8b32c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  retiros_processed_df.drop('Attrition', axis=1, inplace=True)
